#Scrape Data
Note: Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
from datetime import datetime

In [ ]:
def cleanAndSaveSpreadData():
    # Initialize the final DataFrame with the specified columns
    final_df = pd.DataFrame(columns=["Date", "Visitor", "Home", "Home Spread"])

    # Load data from CSV file
    spread_data = pd.read_csv("/content/drive/Shareddrives/CS221 Project/Data/rawData/rawSpreadData.csv")

    for index, row in spread_data.iterrows():
        # Extract the date and convert it into datetime object
        game_date = datetime.strptime(str(row['date']), '%Y%m%d')

        # Extract year from the date
        game_year = game_date.year

        # Add a new row to the final DataFrame
        final_df = final_df.append({
            "Date": game_date,
            "Visitor": row['away'],
            "Home": row['home'],
            "Home Spread": row['home_line'],
        }, ignore_index=True)

    return final_df


In [ ]:
def scrapeScheduleData(years, months):
  final_df = pd.DataFrame(columns = ["Date", "Start (ET)", "Visitor", "Visitor Points", "Home", "Home Points","OT", "Arena","Year"])
  for year in years.keys():
    y = year
    for month in months:
        url = f"https://www.basketball-reference.com/leagues/NBA_{y}_games-{month}.html"
        try:
          html = urlopen(url)
        except:
          html = urlopen(f"https://www.basketball-reference.com/leagues/NBA_{y}_games.html")
        soup = BeautifulSoup(html, features="lxml")
        titles = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
        rows = soup.findAll('tr')[1:]
        team_stats = [[td.getText() for td in rows[i].findAll(['th','td'])] for i in range(len(rows))]
        year_standings = pd.DataFrame(team_stats,columns=["Date", "Start (ET)", "Visitor", "Visitor Points", "Home", "Home Points","Box Score", "OT", "Attend.","Arena", "Notes"])
        year_standings = year_standings.drop(columns = ['Box Score', "Attend.", "Notes"])
        year_standings["Year"] = y
        year_standings = year_standings[year_standings['Date'] != "Playoffs"]
        year_standings['Date'] = year_standings['Date'].map(lambda x : datetime.strptime(x, "%a, %b %d, %Y"))
        year_standings = year_standings[year_standings['Date'] < years[y]]
        final_df = pd.concat([final_df,year_standings])
    final_df.to_csv(f"/content/drive/Shareddrives/CS221 Project/Data/{y}.csv")
  return final_df

In [ ]:
# years = ["2012","2013","2014","2015","2016","2017","2018","2019","2022","2023"]
# start_dates = ["April 27 2012", "April 19 2013", "April 18 2014", "April 17 2015", "April 15 2016", "April 14 2017", "April 13 2018", "April 12 2019", "April 15 2022", "April 14 2023"]
years = ["2023"]
start_dates = ["April 14 2023"]

start_datetimes = [datetime.strptime(data, "%B %d %Y") for data in start_dates]
year_data = {} #Year and corresponding playoff start date
for i in range(len(years)):
  year_data[years[i]] = start_datetimes[i]
months = ["october","november","december","january","february", "march", "april","may"]
#months = ["december"]

In [ ]:
df = scrapeScheduleData(year_data, months)

In [ ]:
len(df)

1234

#Feature Engineering

In [ ]:
from glob import glob
import os

In [ ]:
from math import cos, asin, sqrt, pi


In [ ]:
dir = "/content/drive/Shareddrives/CS221 Project/Data/*.csv"
list_of_csv_files = list(glob(dir))


In [ ]:
df = pd.DataFrame(columns = ["Date", "Start (ET)", "Visitor", "Visitor Points", "Home", "Home Points","OT", "Arena","Year"])
for file in list_of_csv_files:
  dfTemp = pd.read_csv(file)
  df = pd.concat([df, dfTemp])

In [ ]:
df = df.drop_duplicates()

In [ ]:
df = df[["Date", "Start (ET)", "Visitor", "Visitor Points", "Home", "Home Points","OT", "Arena","Year"]]

In [ ]:
df = df.fillna("None")

In [ ]:
df = df[df['Year'] != "None"]

In [ ]:
df['Year'].value_counts()

2022    1234
2023    1234
2014    1230
2015    1230
2016    1230
2018    1230
2019    1230
2017    1230
2013    1229
2012     990
Name: Year, dtype: int64

In [ ]:
df.head()

,Date,Start (ET),Visitor,Visitor Points,Home,Home Points,OT,Arena,Year
0,2011-12-25,12:00p,Boston Celtics,104,New York Knicks,106,None,Madison Square Garden (IV),2012
1,2011-12-25,2:30p,Miami Heat,105,Dallas Mavericks,94,None,American Airlines Center,2012
2,2011-12-25,5:00p,Chicago Bulls,88,Los Angeles Lakers,87,None,STAPLES Center,2012
3,2011-12-25,8:00p,Orlando Magic,89,Oklahoma City Thunder,97,None,Chesapeake Energy Arena,2012
4,2011-12-25,10:30p,Los Angeles Clippers,105,Golden State Warriors,86,None,Oracle Arena,2012


In [ ]:
latLong = pd.read_csv("/content/drive/Shareddrives/CS221 Project/Data/Lat_Long.csv")

In [ ]:
latLong

,Home Team,Arena,Latitude,Longitude
0,Atlanta Hawks,State Farm Arena,33.757° N,84.396° W
1,Boston Celtics,TD Garden,42.366° N,71.062° W
2,Brooklyn Nets,Barclays Center,40.683° N,73.975° W
3,Charlotte Hornets,Spectrum Center,35.225° N,80.839° W
4,Chicago Bulls,United Center,41.881° N,87.675° W
5,Cleveland Cavaliers,Quicken Loans Arena,41.497° N,81.688° W
6,Dallas Mavericks,American Airlines Center,32.791° N,96.81° W
7,Denver Nuggets,Pepsi Center,39.749° N,105.008° W
8,Detroit Pistons,Little Caesars Arena,42.341° N,83.055° W
9,Golden State Warriors,Oracle Arena,37.75° N,122.203° W


In [ ]:
latitudes = dict(zip(latLong['Home Team'], latLong['Latitude']))

In [ ]:
longitudes = dict(zip(latLong['Home Team'], latLong['Longitude']))

In [ ]:
df['Latitude'] = df['Home'].map(latitudes)

In [ ]:
df['Longitude'] = df['Home'].map(longitudes)

In [ ]:
df['Latitude'] = df['Latitude'].fillna("34.043° N")
df['Longitude'] = df['Longitude'].fillna("118.267° W")

In [ ]:
df.to_csv("/content/drive/Shareddrives/CS221 Project/Data/EngineeredScheduleDatav1.csv")

In [ ]:
def distance(lat1, lon1, lat2, lon2):
    r = 6371 # km
    p = pi / 180

    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 2 * r * asin(sqrt(a))

In [ ]:
def getTeamDistances(team, schedule):
  df = pd.concat([schedule[schedule['Visitor']==team][['Date','Latitude','Longitude']],schedule[schedule['Home']==team][['Date','Latitude','Longitude']]])
  df = df.sort_values(by = ['Date'])
  km_traveled = [0.0]
  for i in range(1,len(df)):
    km_traveled.append(distance(float(df.iloc[i-1]['Latitude'][:-3]),float(df.iloc[i-1]['Longitude'][:-3]),float(df.iloc[i]['Latitude'][:-3]),float(df.iloc[i]['Longitude'][:-3])))
  df['km traveled'] = km_traveled
  return df

In [ ]:
df['Home'].unique()

array(['New York Knicks', 'Dallas Mavericks', 'Los Angeles Lakers',
       'Oklahoma City Thunder', 'Golden State Warriors',
       'Cleveland Cavaliers', 'Washington Wizards', 'Orlando Magic',
       'Indiana Pacers', 'Charlotte Bobcats', 'Minnesota Timberwolves',
       'San Antonio Spurs', 'Phoenix Suns', 'Portland Trail Blazers',
       'Sacramento Kings', 'New Jersey Nets', 'Miami Heat',
       'Milwaukee Bucks', 'Toronto Raptors', 'Atlanta Hawks',
       'Detroit Pistons', 'New Orleans Hornets', 'Memphis Grizzlies',
       'Denver Nuggets', 'Houston Rockets', 'Boston Celtics', 'Utah Jazz',
       'Los Angeles Clippers', 'Chicago Bulls', 'Philadelphia 76ers',
       'Brooklyn Nets', 'New Orleans Pelicans', 'Charlotte Hornets'],
      dtype=object)

In [ ]:
teamDistanceDf = {}
for team in df['Home'].unique():
  teamDistanceDf[team] = getTeamDistances(team,df)

In [ ]:
df['Visitor Miles'] = 0.0
for i in range(len(df)):
  date = df['Date'].iloc[i]
  visitingTeam = df['Visitor'].iloc[i]
  df['Visitor Miles'].iloc[i] = teamDistanceDf[visitingTeam].loc[teamDistanceDf[visitingTeam]['Date'] == date]['km traveled'][i]

<ipython-input-130-6a294e70107a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Visitor Miles'].iloc[i] = teamDistanceDf[visitingTeam].loc[teamDistanceDf[visitingTeam]['Date'] == date]['km traveled'][i]


In [ ]:
df['Home Miles'] = 0.0
for i in range(len(df)):
  date = df['Date'].iloc[i]
  homeTeam = df['Home'].iloc[i]
  df['Home Miles'].iloc[i] = teamDistanceDf[homeTeam].loc[teamDistanceDf[homeTeam]['Date'] == date]['km traveled'][i]

<ipython-input-131-1b34b8c0c56b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Home Miles'].iloc[i] = teamDistanceDf[homeTeam].loc[teamDistanceDf[homeTeam]['Date'] == date]['km traveled'][i]


In [ ]:
df = df.drop(columns=['Latitude','Longitude','Year'])

In [ ]:
df.head()

,Date,Start (ET),Visitor,Visitor Points,Home,Home Points,OT,Arena,Visitor Miles,Home Miles
0,2011-12-25,12:00p,Boston Celtics,104,New York Knicks,106,None,Madison Square Garden (IV),0.0,0.0
1,2011-12-25,2:30p,Miami Heat,105,Dallas Mavericks,94,None,American Airlines Center,0.0,0.0
2,2011-12-25,5:00p,Chicago Bulls,88,Los Angeles Lakers,87,None,STAPLES Center,0.0,0.0
3,2011-12-25,8:00p,Orlando Magic,89,Oklahoma City Thunder,97,None,Chesapeake Energy Arena,0.0,0.0
4,2011-12-25,10:30p,Los Angeles Clippers,105,Golden State Warriors,86,None,Oracle Arena,0.0,0.0


In [ ]:
df.to_csv("/content/drive/Shareddrives/CS221 Project/Data/EngineeredScheduleData.csv")